In [ ]:
# define Spark client

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .config(
            "spark.sql.extensions",
            "org.projectnessie.spark.extensions.NessieSparkSessionExtensions, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
        )
        .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .config("spark.sql.catalog.iceberg.type", "rest")
        .config("spark.sql.catalog.iceberg.uri", "http://nessie:19120/iceberg/main/")
        .config("spark.sql.catalog.iceberg.ref", "main")
        .config("spark.sql.catalog.iceberg.cache-enabled", False)
        .getOrCreate()
)

spark.sparkContext.setLogLevel('ERROR')

In [ ]:
# define a view

spark.sql("""
CREATE VIEW iceberg.default.bank_transfers_view (transaction_id COMMENT 'identifier', transaction_amount COMMENT 'amount')
   COMMENT 'View of transactinos'
   AS SELECT id, amount FROM iceberg.default.bank_transfers
""")

In [ ]:
# select from the view 

spark.sql("""
SELECT * FROM iceberg.default.bank_transfers_view
""").show()

In [ ]:
# select all data in the view using Trino

rows = trino.execute("""
SELECT *
FROM iceberg.default.bank_transfers_view
""").fetchall()

for row in rows:
    print(row)